In [1]:
import numpy as np

from standard_tester import *

from runlmc.models.lmc import LMC
from runlmc.kern.rbf import RBF
from runlmc.models.optimization import AdaDelta
from runlmc.models.gpy_lmc import GPyLMC

%matplotlib inline 
import matplotlib.pyplot as plt

np.random.seed(1234)

In [2]:
# as chosen in Nguyen 2014
ks = [RBF(name='rbf0')]
ranks = [2]
# all_ixs is union of all xss, holdout, holdin are dictionaries of slices
xss, yss, test_xss, test_yss, fx2007, all_ixs, holdout, holdin = foreign_exchange_2007()
pred_xss = test_xss

In [3]:
num_runs = 5
inducing_points = np.arange(10, 201, 10)
runlmc_times, runlmc_smses, runlmc_nlpds = [], [], []
for m in inducing_points:
    time, smse, nlpd, _ = runlmc(
        num_runs, m,
        xss, yss, test_xss, test_yss,
        ks, ranks, {'max_it': 10000, 'verbosity':0})
    print('m', m, 'time', time, 'smse', smse, 'nlpd', nlpd)
    runlmc_times.append(time)
    runlmc_smses.append(smse)
    runlmc_nlpds.append(nlpd)

m 10 time 5.79781786233 smse 0.713527479623 nlpd -3.37818285018
m 20 time 10.4916208013 smse 0.471615797868 nlpd -3.50286859391
m 30 time 13.430271755 smse 0.421427986534 nlpd -3.33879070203
m 40 time 11.6561983067 smse 0.255788796642 nlpd -3.91632757374
m 50 time 20.899441047 smse 0.359518176955 nlpd -3.6457910415
m 60 time 31.5921985857 smse 0.445024847426 nlpd -3.29229446597


Process ForkPoolWorker-2178:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
Process ForkPoolWorker-2180:
Process ForkPoolWorker-2179:
Process ForkPoolWorker-2177:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/

lmc: KeyboardInterrupt caught, terminating optimization.


KeyboardInterrupt: 

In [ ]:
cogp_num_runs = 2
cogp_times, cogp_smses, cogp_nlpds = [], [], []
for m in inducing_points:
    time, smse, nlpd = cogp_fx2007(cogp_num_runs, m)
    print('m', m, 'time', time, 'smse', smse, 'nlpd', nlpd)
    cogp_times.append(time)
    cogp_smses.append(smse)
    cogp_nlpds.append(nlpd)

In [ ]:
#also runlmc_nlpds, cogp_nlpds
plt.loglog(runlmc_times, runlmc_smses)
plt.loglog(cogp_times, cogp_smses)
plt.show()

In [ ]:
_, _, _, best = runlmc(
    1, 150,
    xss, yss, test_xss, test_yss,
    ks, ranks, {'max_it': 100, 'verbosity':20})

In [ ]:
# Prints best lmc model
test_fx = [k for k, v in holdout.items() if v.start != v.stop]
pred_xss = [all_ixs if col in test_fx else [] for col in fx2007.columns]
pred_yss, pred_vss = best.predict(pred_xss)
pred_xss = {col: xs for col, xs in zip(fx2007.columns, pred_xss)}
pred_yss = {col: ys for col, ys in zip(fx2007.columns, pred_yss)}
pred_vss = {col: vs for col, vs in zip(fx2007.columns, pred_vss)}

_, axs = plt.subplots(ncols=3, figsize=(16, 4))
for col, ax in zip(test_fx, axs):
    
    # Prediction everywhere
    pred_xs = pred_xss[col]
    pred_ys = pred_yss[col]
    sd = np.sqrt(pred_vss[col])
    ax.plot(pred_xs, pred_ys, c='black')
    top = pred_ys + 2 * sd
    bot = pred_ys - 2 * sd
    ax.fill_between(pred_xs, bot, top, facecolor='grey', alpha=0.3)    
    
    # Actual holdout
    marker_size = 5
    test_xs = all_ixs[holdout[col]]
    test_ys = np.reciprocal(fx2007.ix[holdout[col], col].values)
    ax.scatter(test_xs, test_ys, c='blue', edgecolors='none', s=marker_size, zorder=11)
    
    # Rest of image
    rest_xs = all_ixs[holdin[col]]
    rest_ys = np.reciprocal(fx2007.ix[holdin[col], col].values)
    ax.scatter(rest_xs, rest_ys, c='magenta', edgecolors='none', s=marker_size, zorder=10)
    
    ax.set_xlim([0, 250])
    ax.set_title('output {} (95%)'.format(col))


plt.show()